In [5]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect("retail_customer_experience.db")
cursor = conn.cursor()


## STEP 1 — Create customer_transaction_view

In [3]:

# Create view
cursor.executescript("""
CREATE VIEW IF NOT EXISTS customer_transaction_view AS
SELECT 
    c.customer_id,
    c.full_name,
    c.city,
    c.gender,
    COUNT(t.transaction_id) AS total_transactions,
    SUM(t.price * t.quantity) AS total_spent,
    AVG(t.price * t.quantity) AS avg_transaction_value,
    MAX(t.transaction_date) AS last_purchase_date
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id;
""")

conn.commit()
print(" View 'customer_transaction_view' created successfully.")


 View 'customer_transaction_view' created successfully.


In [9]:
df_txn = pd.read_sql_query("SELECT * FROM customer_transaction_view LIMIT 10;", conn)
df_txn


,customer_id,full_name,city,gender,total_transactions,total_spent,avg_transaction_value,last_purchase_date
0,00012aa8-e99c-4e30-b3f6-1f7e36adc517,Ruth White,Vancouver,Female,5,2734.46,546.892000,2025-01-21
1,000feeed-f931-4908-b539-29ab57e595be,Makayla Hamilton,Durham,Female,6,2813.06,468.843333,2024-11-23
2,00107f62-0530-48ec-a56a-49d90944eafe,Jason Willis,Tacoma,Male,1,161.99,161.990000,2024-12-13
3,005ae094-2f68-4a6d-91f0-73b38b890692,Gary Mack,Rockford,Male,6,3691.46,615.243333,2024-12-17
4,00a6835b-c50a-49a9-a1b5-19d74d3e1863,William Sparks,Fort Worth,Male,7,3820.98,545.854286,2025-01-31
5,00a90d43-e62f-4e16-9a17-948d25c9684b,Tina Gilbert,Lansing,Female,7,5636.89,805.270000,2024-12-28
6,00ca767b-0217-46ba-b482-021916fa1be0,Andrew Adams,Houston,Male,2,220.59,110.295000,2025-01-23
7,00ca868d-4a6a-47c1-9231-a8279337da2f,Victoria Mcclain,San Francisco,Female,0,NaN,NaN,None
8,00df5d7c-c28c-436c-aa68-e3ceb775fff2,Alexander Lee,San Diego,Male,5,6560.97,1312.194000,2025-01-28
9,01028498-4406-4014-a30c-2b9a605c2bfc,Andrew Evans,San Jose,Male,6,5507.84,917.973333,2024-11-15


## STEP 2 — Create customer_support_view

In [12]:
cursor.executescript("""
CREATE VIEW IF NOT EXISTS customer_support_view AS
SELECT 
    c.customer_id,
    COUNT(s.ticket_id) AS total_tickets,
    AVG(s.resolution_time_hours) AS avg_resolution_time,
    AVG(s.customer_satisfaction_score) AS avg_support_score
FROM customers c
LEFT JOIN support_tickets s ON c.customer_id = s.customer_id
GROUP BY c.customer_id;
""")

conn.commit()
print(" View 'customer_support_view' created successfully.")


 View 'customer_support_view' created successfully.


In [13]:
df_support = pd.read_sql_query("SELECT * FROM customer_support_view LIMIT 10;", conn)
df_support.head()


,customer_id,total_tickets,avg_resolution_time,avg_support_score
0,00012aa8-e99c-4e30-b3f6-1f7e36adc517,0,NaN,NaN
1,000feeed-f931-4908-b539-29ab57e595be,0,NaN,NaN
2,00107f62-0530-48ec-a56a-49d90944eafe,0,NaN,NaN
3,005ae094-2f68-4a6d-91f0-73b38b890692,1,41.0,2.0
4,00a6835b-c50a-49a9-a1b5-19d74d3e1863,0,NaN,NaN


## STEP 3 — Create customer_review_view

In [14]:
cursor.executescript("""
CREATE VIEW IF NOT EXISTS customer_review_view AS
SELECT 
    c.customer_id,
    COUNT(r.review_id) AS total_reviews,
    AVG(r.rating) AS avg_rating,
    MAX(r.review_date) AS last_review_date
FROM customers c
LEFT JOIN customer_reviews r ON c.customer_id = r.customer_id
GROUP BY c.customer_id;
""")

conn.commit()
print(" View 'customer_review_view' created successfully.")


 View 'customer_review_view' created successfully.


In [15]:
df_reviews = pd.read_sql_query("SELECT * FROM customer_review_view LIMIT 10;", conn)
df_reviews.head()


,customer_id,total_reviews,avg_rating,last_review_date
0,00012aa8-e99c-4e30-b3f6-1f7e36adc517,1,4.0,2023-11-10
1,000feeed-f931-4908-b539-29ab57e595be,0,NaN,None
2,00107f62-0530-48ec-a56a-49d90944eafe,0,NaN,None
3,005ae094-2f68-4a6d-91f0-73b38b890692,0,NaN,None
4,00a6835b-c50a-49a9-a1b5-19d74d3e1863,1,4.0,2023-03-25


## STEP 4 — Create customer_360_view (Master View)

In [17]:
cursor.executescript("""
CREATE VIEW IF NOT EXISTS customer_360_view AS
SELECT 
    c.customer_id,
    c.full_name,
    c.city,
    c.gender,
    c.age,
    c.preferred_channel,
    ct.total_transactions,
    ct.total_spent,
    ct.avg_transaction_value,
    ct.last_purchase_date,
    cs.total_tickets,
    cs.avg_resolution_time,
    cs.avg_support_score,
    cr.total_reviews,
    cr.avg_rating,
    cr.last_review_date
FROM customers c
LEFT JOIN customer_transaction_view ct ON c.customer_id = ct.customer_id
LEFT JOIN customer_support_view cs ON c.customer_id = cs.customer_id
LEFT JOIN customer_review_view cr ON c.customer_id = cr.customer_id;
""")

conn.commit()
print("Master view 'customer_360_view' created successfully.")


Master view 'customer_360_view' created successfully.


In [18]:
df_customer_360 = pd.read_sql_query("SELECT * FROM customer_360_view LIMIT 10;", conn)
df_customer_360


,customer_id,full_name,city,gender,age,preferred_channel,total_transactions,total_spent,avg_transaction_value,last_purchase_date,total_tickets,avg_resolution_time,avg_support_score,total_reviews,avg_rating,last_review_date
0,4c30e132-0704-4459-a509-9eddde934977,Mark Johnson,Houston,Male,40.0,None,3,3565.72,1188.573333,2024-12-17,1,45.0,5.0,0,NaN,None
1,68bec407-275f-4b5b-9a82-13d02f54626a,Robert Smith,Austin,Male,33.0,in-store,9,8925.78,991.753333,2024-09-16,0,NaN,NaN,0,NaN,None
2,4466459f-76c8-433c-814e-6d59cb4131fc,Jamie Chavez,Detroit,Female,42.0,online,4,3820.41,955.102500,2025-02-14,0,NaN,NaN,0,NaN,None
3,04c36a25-02f3-462c-92b0-6bf291c57706,Thomas Bradley,Fort Worth,Male,53.0,both,6,1883.76,376.752000,2024-12-28,1,30.0,5.0,0,NaN,None
4,e916df3d-c3f5-40b0-8ae2-5d043be88300,Jane Ferrell,Atlanta,Female,32.0,online,15,8161.57,582.969286,2024-10-18,0,NaN,NaN,0,NaN,None
5,0709fd3c-d251-486e-8864-4b18dcd01e57,Jacqueline Martin,San Francisco,Female,32.0,in-store,6,5837.17,972.861667,2025-01-25,0,NaN,NaN,0,NaN,None
6,d7c50fae-43b8-4945-86f8-b8f9d4d2d71a,Jennifer Powers,Tempe,Female,53.0,in-store,6,3361.67,560.278333,2024-12-19,0,NaN,NaN,0,NaN,None
7,d61d8f7b-c2d0-4548-9f3b-da16dc3146ad,David Kennedy,Tucson,Male,44.0,in-store,5,3397.99,679.598000,2024-10-24,1,43.0,4.0,0,NaN,None
8,fd54988d-3cca-4a02-9254-b4fa7a2d5451,Anthony Kidd,New York City,Male,29.0,online,1,1122.26,1122.260000,2024-07-10,1,5.0,4.0,0,NaN,None
9,a960dd54-ee35-4731-b29e-d8de6c5793dc,Jonathan Farrell,Syracuse,Male,41.0,in-store,10,11480.77,1275.641111,2024-09-07,1,15.0,4.0,1,5.0,2023-02-16


In [19]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='view';")
print("Available Views:", cursor.fetchall())


Available Views: [('customer_transaction_view',), ('customer_support_view',), ('customer_review_view',), ('customer_360_view',)]


In [20]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='view';")
print("Available Views:", cursor.fetchall())


Available Views: [('customer_transaction_view',), ('customer_support_view',), ('customer_review_view',), ('customer_360_view',)]
